In [1]:
from IPython.display import Image, display

from langchain_mcp_adapters.client import MultiServerMCPClient

from react_agent.src.agents.react_agent import ReActAgent

from react_agent.src.util.llm_proxy import LLMProxy

In [2]:
WITH_ML_FLOW = False

In [3]:
# Configure the MCP Server for tool consumption
from react_agent.src.config.system_parameters import QAToolsServerSettings
QA_SERVER_SETTINGS = QAToolsServerSettings()

multiServerClientConfig = {
    "QuestionAnsweringTools": {
        "url": f"http://{QA_SERVER_SETTINGS.host}:{QA_SERVER_SETTINGS.port}/{QA_SERVER_SETTINGS.transport}",
        "transport": QA_SERVER_SETTINGS.transport,
    }
}

In [4]:
if(WITH_ML_FLOW):
    import mlflow
    # poetry run mlflow server --host 127.0.0.1 --port 8080
    mlflow.set_tracking_uri(uri="http://localhost:8080")
    mlflow.set_experiment("Testing of email ReAct Agent")

    mlflow.langchain.autolog()

In [5]:
mapping_question = "Explain how 'Payment Terms' is mapped. Start with 'map_invoice1'."
extensibility_question = "As a Public Cloud customer in Spain, can I extend an existing eDocument customer invoice Process?"
peppol_support = """We as Syntax are currently implementing SAP DRC and are following the official SAP help.
We are currently at the step “Integration of Peppol Exchange with SAP S/4HANA Cloud” and there at the creation of an S/4HANA Public Cloud communication user.
A client certificate should be uploaded for the communication user to be created, as Basic Auth (via user + password) is not supported.

The problem: We do not have this certificate. Where can I get this certificate? What format must it be in?
We are stuck at this step because we are missing the certificate mentioned and it is also not clear to us what kind of certificate is meant here and where to get it.
"""

In [6]:
async with MultiServerMCPClient( multiServerClientConfig) as client:
    agent = ReActAgent(tool_list=client.get_tools())
    print(agent.create_sys_prompt())
    # display(Image(agent.get_agent_graph()))

    await agent.arun_and_print_agent_stream(user_message=extensibility_question)


< Role >
You are an expert on Electronic Document Processing.
</ Role >

< Instructions >
Context: Peppol, UBL, eInvoicing.
1. Begin with an observation that outlines the primary task or question you want the agent to address.
2. Analyze the observation to generate exactly one thought that leads to an actionable step using one of the available tools.
3. Log the generated thought and corresponding action pair for transparency and future reference.
4. Execute the exactly one action using the choosen tool and specify the parameters needed.
5. Collect the new observation or insights generated from the tool's output.
6. Is further analysis or action needed, think how other possible tools may help to improve the output?
        - If yes, create new thought and action pairs.
        - If no, provide a concise conclusion.
Avoid bias based on physical appearance, ethnicity, or race.
Replace inappropriate language with inclusive language; politely refuse results, if that is not possible.
</ Ins

In [7]:
# print(agent.response)

In [8]:
LLMProxy().print_usage()

Call Count: 0
Token Usage: {'input_tokens': 0, 'output_tokens': 0, 'total_tokens': 0}
